In [1]:
import numpy as np
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout,MaxPooling1D
from keras.optimizers import SGD
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_excel('completeEncodedAndScaledDataset.xlsx')
df.head()


,Protocol_type,Service,Flag,Src_bytes,Logged_in,Count,Srv_count,Serror_rate,Srv_serror_rate,Rerror_rate,...,Dst_host_srv_count,Dst_host_same_srv_rate,Dst_host_diff_srv_rate,Dst_host_same_src_port_rate,Dst_host_srv_diff_host_rate,Dst_host_serror_rate,Dst_host_srv_serror_rate,Dst_host_rerror_rate,attack_type,attack_type_encoded
0,2.038571,0.629977,0.735369,-0.011190,-0.732914,-0.581217,-0.367299,-0.556584,-0.552030,-0.460806,...,-1.050270,-1.193023,2.187298,2.137976,-0.263701,-0.561390,-0.549814,-0.487765,normal,16
1,-0.154786,0.938904,-0.660992,-0.011262,-0.732914,0.275339,-0.312117,1.851192,1.851769,-0.460806,...,-0.821669,-0.966271,-0.237144,-0.498320,-0.263701,1.904034,1.899674,-0.487765,neptune,14
2,-0.154786,-0.543948,0.735369,-0.011147,1.364417,-0.643512,-0.323153,-0.075029,-0.071270,-0.460806,...,1.272317,1.074493,-0.457548,-0.408446,0.094049,-0.487427,-0.525319,-0.487765,normal,16
3,-0.154786,-0.543948,0.735369,-0.011163,1.364417,-0.448840,-0.025167,-0.556584,-0.552030,-0.460806,...,1.272317,1.074493,-0.457548,-0.498320,-0.263701,-0.561390,-0.549814,-0.487765,normal,16
4,-0.154786,0.938904,-2.057353,-0.011262,-0.732914,0.259766,-0.168642,-0.556584,-0.552030,2.274941,...,-0.885678,-1.034297,-0.148983,-0.498320,-0.263701,-0.561390,-0.549814,2.476323,neptune,14


In [3]:
df=df.drop(columns=['attack_type'])
df

,Protocol_type,Service,Flag,Src_bytes,Logged_in,Count,Srv_count,Serror_rate,Srv_serror_rate,Rerror_rate,...,Dst_host_count,Dst_host_srv_count,Dst_host_same_srv_rate,Dst_host_diff_srv_rate,Dst_host_same_src_port_rate,Dst_host_srv_diff_host_rate,Dst_host_serror_rate,Dst_host_srv_serror_rate,Dst_host_rerror_rate,attack_type_encoded
0,2.038571,0.629977,0.735369,-0.011190,-0.732914,-0.581217,-0.367299,-0.556584,-0.552030,-0.460806,...,0.656445,-1.050270,-1.193023,2.187298,2.137976,-0.263701,-0.561390,-0.549814,-0.487765,16
1,-0.154786,0.938904,-0.660992,-0.011262,-0.732914,0.275339,-0.312117,1.851192,1.851769,-0.460806,...,0.656445,-0.821669,-0.966271,-0.237144,-0.498320,-0.263701,1.904034,1.899674,-0.487765,14
2,-0.154786,-0.543948,0.735369,-0.011147,1.364417,-0.643512,-0.323153,-0.075029,-0.071270,-0.460806,...,-1.709884,1.272317,1.074493,-0.457548,-0.408446,0.094049,-0.487427,-0.525319,-0.487765,16
3,-0.154786,-0.543948,0.735369,-0.011163,1.364417,-0.448840,-0.025167,-0.556584,-0.552030,-0.460806,...,0.656445,1.272317,1.074493,-0.457548,-0.498320,-0.263701,-0.561390,-0.549814,-0.487765,16
4,-0.154786,0.938904,-2.057353,-0.011262,-0.732914,0.259766,-0.168642,-0.556584,-0.552030,2.274941,...,0.656445,-0.885678,-1.034297,-0.148983,-0.498320,-0.263701,-0.561390,-0.549814,2.476323,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37036,2.038571,-1.223588,0.735369,-0.011241,-0.732914,0.532306,2.226285,-0.556584,-0.552030,-0.460806,...,0.656445,1.272317,1.074493,-0.457548,-0.468362,-0.263701,-0.561390,-0.549814,-0.487765,16
37037,-0.154786,-0.543948,0.735369,-0.011096,1.364417,-0.612365,-0.267971,-0.556584,-0.552030,-0.460806,...,0.656445,1.080292,0.893092,-0.369387,-0.498320,-0.263701,-0.561390,-0.549814,-0.339561,16
37038,-0.154786,1.618544,-1.708262,-0.011262,-0.732914,-0.674660,-0.301080,-0.556584,-0.215498,2.274941,...,-0.163882,-0.181586,-0.354042,-0.325306,-0.468362,-0.084826,-0.438119,-0.353855,2.031710,11
37039,-0.154786,1.433188,-2.057353,-0.011262,-0.732914,-0.643512,-0.323153,-0.556584,-0.552030,1.727792,...,-0.226984,-0.583924,-0.762195,-0.325306,-0.468362,0.094049,-0.561390,-0.549814,2.120633,11


In [36]:
##DATA FOR PREDICTION
df.iloc[4].values.tolist()

[-0.1547861708316782,
 0.9389039675911617,
 -2.057352539144836,
 -0.01126189644544677,
 -0.7329135708671861,
 0.2597657205824371,
 -0.1686418959758708,
 -0.5565841949836169,
 -0.5520301935421467,
 2.274941066835758,
 -1.235048984171417,
 -0.1330656931473705,
 0.6564451117594703,
 -0.8856776540511395,
 -1.034296750645344,
 -0.1489828029008353,
 -0.4983199439537688,
 -0.2637008691016175,
 -0.561389892449471,
 -0.5498138592727534,
 2.476323322978251,
 14.0]

In [5]:
# split into input (X) and output (Y) variables
X = df.drop(columns=['attack_type_encoded'])
Y = df[['attack_type_encoded']]
print(X)
print(Y)




       Protocol_type   Service      Flag  Src_bytes  Logged_in     Count  \
0           2.038571  0.629977  0.735369  -0.011190  -0.732914 -0.581217   
1          -0.154786  0.938904 -0.660992  -0.011262  -0.732914  0.275339   
2          -0.154786 -0.543948  0.735369  -0.011147   1.364417 -0.643512   
3          -0.154786 -0.543948  0.735369  -0.011163   1.364417 -0.448840   
4          -0.154786  0.938904 -2.057353  -0.011262  -0.732914  0.259766   
...              ...       ...       ...        ...        ...       ...   
37036       2.038571 -1.223588  0.735369  -0.011241  -0.732914  0.532306   
37037      -0.154786 -0.543948  0.735369  -0.011096   1.364417 -0.612365   
37038      -0.154786  1.618544 -1.708262  -0.011262  -0.732914 -0.674660   
37039      -0.154786  1.433188 -2.057353  -0.011262  -0.732914 -0.643512   
37040       2.038571  0.938904  0.735369  -0.011242  -0.732914 -0.651299   

       Srv_count  Serror_rate  Srv_serror_rate  Rerror_rate  ...  \
0      -0.367299   

In [6]:
nb_features=21 #input parameters
nb_class=40 #output classes


In [7]:
Y = to_categorical(Y, nb_class)

In [8]:
print(X.shape)
print(Y.shape)


(37041, 21)
(37041, 40)


In [9]:
# reshape data
X = np.expand_dims(X,axis=2)
print(X.shape)

(37041, 21, 1)


In [10]:
Y.shape

(37041, 40)

In [11]:
trainx, testx, trainy, testy = train_test_split(X,Y, test_size=0.25, random_state=42)

In [12]:
print(trainx.shape)
print(trainy.shape)
print(testx.shape)
print(testy.shape)

(27780, 21, 1)
(27780, 40)
(9261, 21, 1)
(9261, 40)


In [13]:
nb_timesteps=trainx.shape[1]
nb_features=trainx.shape[2]

In [18]:
model = Sequential()
model.add(Convolution1D(filters=64, kernel_size=1, input_shape=(nb_timesteps,nb_features)))
model.add(Convolution1D(filters=64, kernel_size=1))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.1))
model.add(Dense(100, activation='relu'))
model.add(Dense(nb_class,activation="softmax"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 21, 64)            128       
                                                                 
 conv1d_5 (Conv1D)           (None, 21, 64)            4160      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 10, 64)           0         
 1D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 640)               0         
                                                                 
 dropout_2 (Dropout)         (None, 640)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               64100     
                                                      

In [19]:
#sgd = SGD(lr=0.01, nesterov=True, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(trainx, trainy, epochs=25, validation_data=(testx, testy),batch_size=64)


Epoch 1/25
435/435 [==============================] - 10s 22ms/step - loss: 0.6157 - accuracy: 0.8335 - val_loss: 0.3927 - val_accuracy: 0.8943
Epoch 2/25
435/435 [==============================] - 10s 23ms/step - loss: 0.3310 - accuracy: 0.8972 - val_loss: 0.2866 - val_accuracy: 0.9109
Epoch 3/25
435/435 [==============================] - 23s 54ms/step - loss: 0.2743 - accuracy: 0.9100 - val_loss: 0.2870 - val_accuracy: 0.9049
Epoch 4/25
435/435 [==============================] - 3s 8ms/step - loss: 0.2555 - accuracy: 0.9174 - val_loss: 0.2456 - val_accuracy: 0.9182
Epoch 5/25
435/435 [==============================] - 29s 66ms/step - loss: 0.2356 - accuracy: 0.9226 - val_loss: 0.2525 - val_accuracy: 0.9207
Epoch 6/25
435/435 [==============================] - 20s 46ms/step - loss: 0.2189 - accuracy: 0.9265 - val_loss: 0.2502 - val_accuracy: 0.9251
Epoch 7/25
435/435 [==============================] - 21s 49ms/step - loss: 0.2121 - accuracy: 0.9289 - val_loss: 0.2212 - val_accuracy: 0

KeyboardInterrupt: 

In [20]:
model.save('1dcnn_model.h5')

In [21]:
import tensorflow

In [22]:
result=['apache2','back', 'buffer_overflow', 'ftp_write', 'guess_passwd',
 'httptunnel', 'imap', 'ipsweep', 'land', 'loadmodule', 'mailbomb', 'mscan',
 'multihop', 'named', 'neptune', 'nmap', 'normal', 'perl', 'phf', 'pod',
 'portsweep', 'processtable', 'ps', 'rootkit', 'saint', 'satan', 'sendmail',
 'smurf', 'snmpgetattack', 'snmpguess', 'spy', 'sqlattack', 'teardrop',
 'udpstorm', 'warezclient', 'warezmaster', 'worm', 'xlock', 'xsnoop', 'xterm']


In [37]:
model =tensorflow.keras.models.load_model('1dcnn_model.h5')

test_data = np.array([[-0.1547861708316782,
 0.9389039675911617,
 -2.057352539144836,
 -0.01126189644544677,
 -0.7329135708671861,
 0.2597657205824371,
 -0.1686418959758708,
 -0.5565841949836169,
 -0.5520301935421467,
 2.274941066835758,
 -1.235048984171417,
 -0.1330656931473705,
 0.6564451117594703,
 -0.8856776540511395,
 -1.034296750645344,
 -0.1489828029008353,
 -0.4983199439537688,
 -0.2637008691016175,
 -0.561389892449471,
 -0.5498138592727534,
 2.476323322978251]])


o=model.predict(test_data.reshape(-1,21,1), batch_size=1)

print(o)
print(len(o[0]))
print(o.argmax())
print(result[int(o.argmax())])

1/1 [==============================] - 0s 77ms/step
[[2.4850719e-10 2.8649836e-11 3.1473307e-15 8.3023882e-10 1.2081673e-06
  2.6865590e-09 8.4107832e-11 2.7827188e-10 5.1591909e-14 7.4629145e-16
  5.9638097e-18 5.8985074e-07 5.0639312e-14 1.4977983e-13 9.9999022e-01
  5.1826538e-18 2.8257266e-06 1.2290463e-13 2.1492426e-15 1.5153770e-19
  1.6300081e-10 2.1092096e-14 1.3976068e-13 4.5060695e-13 1.3792184e-09
  5.1431134e-06 2.9153388e-15 6.4274461e-17 1.5858595e-14 7.0402632e-17
  4.5440793e-17 9.8133687e-13 1.1237746e-15 1.9511812e-13 2.5896431e-16
  7.7934798e-11 1.7161038e-14 1.6142010e-14 8.7824753e-14 2.1952399e-14]]
40
14
neptune
